In [1]:
from newspaper import Article
import pickle as pkl
import pandas as pd
from tqdm import tqdm
import os
from pathlib import Path

In [3]:
with open("nba_links_news2.pkl", "rb") as f:
    links = pkl.load(f)

links = list(set(links))

In [9]:
## Importante manter o padrão no nome para o salvamento automático
dataset_name = "tênnis_news.csv"

Path(dataset_name).mkdir(parents=True, exist_ok=True)

news = pd.read_csv(dataset_name)

for link in tqdm(links, f"{dataset_name.split('_')[0]} News"):
    article = Article(link)
    article.download()
    article.parse()

    authors = article.authors
    data = article.publish_date
    title = article.title
    content = article.text

    news.loc[news.shape[0]] = [title, authors, data, content]
    news.to_csv(dataset_name, index=False)

    del article

SAML News: 100%|██████████| 285/285 [07:07<00:00,  1.50s/it]


## Exemplo de uso da Biblioteca:

In [4]:
url = "https://www.slamonline.com/news/nba/new-off-broadway-play-king-james-examines-how-lebrons-time-in-cleveland-helped-bring-two-strangers-together/"
article = Article(url)

In [5]:
article.download()
article.parse()

In [7]:
article.title

'“King James” Play Examines How LeBron Helped Bring Two Strangers Together'

In [6]:
article.authors

['Brooks Warren',
 'Haben Fessehazion',
 'Drew Ruiz',
 'Deyscha Smith',
 'Slam Staff',
 'Ryan Jones',
 'Ian Pierno']

In [7]:
article.publish_date

datetime.datetime(2023, 6, 8, 17, 5, 32, tzinfo=tzoffset(None, -14400))

In [8]:
article.text

'“King James,” the new off-Broadway play, has taken over NYC in recent weeks. The production examines how sports fandom can bring people from different walks of life together and even spark friendships that otherwise may never be formed. In this case, it focuses on two young men in Cleveland who meet right as LeBron James is entering his rookie season with the Cleveland Cavaliers in 2003. The play proceeds to study how their friendship evolves throughout the years, and even hits a few bumps on the road, all while LeBron’s career is simultaneously evolving—the play starts in 2003 but ends over a decade later, following LeBron from Cleveland to Miami and back to Cleveland.\n\nChris Perfetti (Abbott Elementary, Gotham, Blue Bloods) plays Matt and Glenn Davis (actor, producer and Artistic Director of Steppenwolf Theatre Company; The Christians, Head of Passes, The Brother/Sister Plays, Downstate) plays Shawn. Collectively they perfectly (serious when it needs to be, and hilarious just as a